In [ ]:
# default_exp ai_platform_runner

    echo "Submitting an AI Platform job..."
    # see https://cloud.google.com/sdk/gcloud/reference/ai-platform/jobs/submit/training
    gcloud beta ai-platform jobs submit training ${JOB_NAME} \
            ${CONFIG_FIX} \
            --scale-tier=${SCALE_TIER} \
            --region=${REGION} \
            --master-image-uri=${IMAGE_URI} \
            --stream-logs \
            ${CONFIG} \
            -- python trainer/trainer.py --job-dir=${MODEL_DIR} --train-location=cloud $@

In [ ]:
from gcp_runner import core
core.export_and_reload_all()

In [ ]:
#export
import inspect 
from gcp_runner import ai_platform_constants

def _get_not_none(arg, default):
    return default if arg is None else arg

def _get_common_args(
    job_dir,
    job_name=None,
    region=None,
    scale_tier: ai_platform_constants.ScaleTier = ai_platform_constants.ScaleTier.BASIC,
    master_machine_type: ai_platform_constants.MachineType = None,
    master_image_uri=None,
    master_accelerator_type: ai_platform_constants.AcceleratorType = None,
    master_accelerator_count=None,
    parameter_machine_type: ai_platform_constants.MachineType = None,
    parameter_machine_count=None,
    parameter_image_uri=None,
    parameter_accelerator_type: ai_platform_constants.AcceleratorType = None,
    parameter_accelerator_count=None,
    worker_machine_type: ai_platform_constants.MachineType = None,
    worker_machine_count=None,
    worker_image_uri=None,
    work_accelerator_type: ai_platform_constants.AcceleratorType = None,
    work_accelerator_count=None,
    use_chief_in_tf_config=True,
    distribution_strategy_type: ai_platform_constants.DistributionStrategyType = None):   
    args = []
    
    if distribution_strategy_type and scale_tier == ai_platform_constants.ScaleTier.CUSTOM:
        master_machine_type = _get_not_none(master_machine_type, ai_platform_constants.MachineType.N1_STANDARD_4)
        if distribution_strategy_type == ai_platform_constants.DistributionStrategyType.MIRRORED_STRATEGY:
            master_accelerator_type = _get_not_none (master_accelerator_type, ai_platform_constants.AcceleratorType.NVIDIA_TESLA_K80)
            master_accelerator_count = _get_not_none(master_accelerator_count, 2)
            worker_machine_count = _get_not_none(worker_machine_count, 0)
        elif distribution_strategy_type == ai_platform_constants.DistributionStrategyType.CENTRAL_STORAGE_STRATEGY:
            master_accelerator_type = _get_not_none (master_accelerator_type, ai_platform_constants.AcceleratorType.NVIDIA_TESLA_K80)
            master_accelerator_count = _get_not_none(master_accelerator_count, 2)
            worker_machine_count = _get_not_none(worker_machine_count, 0)
        elif distribution_strategy_type == ai_platform_constants.DistributionStrategyType.PARAMETER_SERVERSTRATEGY:
            parameter_machine_type = _get_not_none(parameter_machine_type, ai_platform_constants.MachineType.N1_STANDARD_4)
            parameter_machine_count = _get_not_none(parameter_machine_count, 1)
            worker_machine_count = _get_not_none(worker_machine_count, 2)
        elif args.distribution_strategy == ai_platform_constants.DistributionStrategyType.MULTI_WORKER_MIRRORED_STRATEGY:
            master_accelerator_type = _get_not_none (master_accelerator_type, ai_platform_constants.AcceleratorType.NVIDIA_TESLA_K80)
            master_accelerator_count = _get_not_none(master_accelerator_count, 2)
            worker_machine_count = _get_not_none(worker_machine_count, 2)
            work_accelerator_type = _get_not_none (work_accelerator_type, ai_platform_constants.AcceleratorType.NVIDIA_TESLA_K80)
            worker_machine_type = _get_not_none(worker_machine_type, ai_platform_constants.MachineType.N1_STANDARD_4)
    
    for key, value in locals().items():
        # print(key + '=' + str(value))
        if key != 'args' and key != 'distribution_strategy_type' and value:
            if key.endswith('_type') or key == 'scale_tier':
                args.append("--%s=%s" % (key.replace('_', '-'), value.value))
            else:
                args.append("--%s=%s" % (key.replace('_', '-'), value))
    
    return args 

Converted 00_core.ipynb.
Converted ai_platform_constants.ipynb.
Converted ai_platform_runner.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted local_runner.ipynb.
Converted sample_code.ipynb.


In [ ]:
#export
import datetime
from gcp_runner.core import build_and_push_docker_image, run_process, get_run_python_args
from gcp_runner import ai_platform_constants

def run_docker_image(
    func, 
    job_dir,
    build_docker_file=None,
    dry_run=False,
    job_name=None,
    region=None,
    scale_tier: ai_platform_constants.ScaleTier =  ai_platform_constants.ScaleTier.BASIC,
    master_machine_type: ai_platform_constants.MachineType = None,
    master_image_uri=None,
    master_accelerator_type: ai_platform_constants.AcceleratorType = None,
    master_accelerator_count=None,
    parameter_machine_type: ai_platform_constants.MachineType = None,
    parameter_machine_count=None,
    parameter_image_uri=None,
    parameter_accelerator_type: ai_platform_constants.AcceleratorType = None,
    parameter_accelerator_count=None,
    worker_machine_type: ai_platform_constants.MachineType = None,
    worker_machine_count=None,
    worker_image_uri=None,
    work_accelerator_type: ai_platform_constants.AcceleratorType = None,
    work_accelerator_count=None,
    use_chief_in_tf_config=True,
    distribution_strategy_type: ai_platform_constants.DistributionStrategyType = None,
    **kwargs):
    
    if not master_image_uri:
        raise ValueError('master_image_uri argument is required')
        
    if not job_name:
        job_name = 'ai_platform_runner_train_docker_' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        
    if build_docker_file is not None:
        result = build_and_push_docker_image(build_docker_file, master_image_uri, dry_run=dry_run)
        if result:
            return result
    
    # see https://cloud.google.com/sdk/gcloud/reference/ai-platform/jobs/submit/training
    args = ['gcloud', 'ai-platform', 'jobs', 'submit', 'training', job_name,
           "--stream-logs"]
    
    common_args = _get_common_args(
        job_dir,
        region = region,
        scale_tier = scale_tier,
        master_machine_type = master_machine_type,
        master_image_uri = master_image_uri,
        master_accelerator_type = master_accelerator_type,
        master_accelerator_count = master_accelerator_count,
        parameter_machine_type = parameter_machine_type,
        parameter_machine_count = parameter_machine_count,
        parameter_image_uri = parameter_image_uri,
        parameter_accelerator_type = parameter_accelerator_type,
        parameter_accelerator_count = parameter_accelerator_count,
        worker_machine_type = worker_machine_type,
        worker_machine_count = worker_machine_count,
        worker_image_uri = worker_image_uri,
        work_accelerator_type = work_accelerator_type,
        work_accelerator_count = work_accelerator_count,
        use_chief_in_tf_config = use_chief_in_tf_config,
        distribution_strategy_type = distribution_strategy_type)
    args.extend(common_args)
    print(common_args)
    print(master_image_uri)
    
    args.append('--')
    
    run_python_args = get_run_python_args(func, **kwargs)
    args.extend(run_python_args)
    args.append("--job-dir=%s" % job_dir)
    if distribution_strategy_type:
        args.append("--distribution-strategy-type=%s" % distribution_strategy_type)
    
    print('running training job using Docker image Google Cloud Platform AI:')
    print(' '.join(args).replace(' --', '\n --').replace('\n', ' \\ \n'))
    if not dry_run:
        return gcp_runner.local_runner.run_process(args)


In [ ]:
import datetime
datetime.datetime.now().strftime('%Y%m%d_%H%M%S')


'20200325_163837'

echo "Submitting an AI Platform job..."
# see https://cloud.google.com/sdk/gcloud/reference/ai-platform/jobs/submit/training
gcloud ai-platform jobs submit training ${JOB_NAME} \
        --config=${PWD}/scripts/config_fix.yaml \
        --scale-tier=${SCALE_TIER} \
        --job-dir=${MODEL_DIR} \
        --runtime-version=${RUNTIME_VERSION} \
        --region=${REGION} \
        --module-name=trainer.trainer \
        --package-path=${PACKAGE_PATH}  \
        --master-machine-type=n1-highcpu-16 \
        --stream-logs \
        ${CONFIG} \
        -- \
        --job-dir=${MODEL_DIR} --train-location=cloud $@

In [ ]:
#export
import os
import datetime
from gcp_runner.core import get_run_python_args, run_process, get_package_name, get_module_name
from gcp_runner import ai_platform_constants

#IMPORTANT: we'll need to either copy entry_point.py to package,
# or tell user to declare it and call into similar entry_point logic manually.

def run_package(
    func,
    job_dir,
    job_name=None,
    runtime_version='2.1', 
    python_version='3.7', 
    dry_run=False,
    region=None,
    scale_tier: ai_platform_constants.ScaleTier =  ai_platform_constants.ScaleTier.BASIC,
    master_machine_type: ai_platform_constants.MachineType = None,
    master_image_uri=None,
    master_accelerator_type: ai_platform_constants.AcceleratorType = None,
    master_accelerator_count=None,
    parameter_machine_type: ai_platform_constants.MachineType = None,
    parameter_machine_count=None,
    parameter_image_uri=None,
    parameter_accelerator_type: ai_platform_constants.AcceleratorType = None,
    parameter_accelerator_count=None,
    worker_machine_type: ai_platform_constants.MachineType = None,
    worker_machine_count=None,
    worker_image_uri=None,
    work_accelerator_type: ai_platform_constants.AcceleratorType = None,
    work_accelerator_count=None,
    use_chief_in_tf_config=True,
    distribution_strategy_type: ai_platform_constants.DistributionStrategyType = None,    
    **kwargs):
    
    if not job_name:
        job_name = 'ai_platform_runner_train_package_' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # see https://cloud.google.com/sdk/gcloud/reference/ai-platform/jobs/submit/training
    package_name = get_package_name()
    module_name = get_module_name()
    function_name = func.__name__
    args = [
        'gcloud', 'ai-platform', 'jobs', 'submit', 'training', job_name,
        "--runtime-version=%s" % runtime_version,
        "--python-version=%s" % python_version,
        "--stream-logs",
        "--module-name=%s.entry_point" % package_name,
        "--package-path=%s/%s" % (os.getcwd(), package_name)]

    common_args = _get_common_args(
        job_dir,
        region = region,
        scale_tier = scale_tier,
        master_machine_type = master_machine_type,
        master_image_uri = master_image_uri,
        master_accelerator_type = master_accelerator_type,
        master_accelerator_count = master_accelerator_count,
        parameter_machine_type = parameter_machine_type,
        parameter_machine_count = parameter_machine_count,
        parameter_image_uri = parameter_image_uri,
        parameter_accelerator_type = parameter_accelerator_type,
        parameter_accelerator_count = parameter_accelerator_count,
        worker_machine_type = worker_machine_type,
        worker_machine_count = worker_machine_count,
        worker_image_uri = worker_image_uri,
        work_accelerator_type = work_accelerator_type,
        work_accelerator_count = work_accelerator_count,
        use_chief_in_tf_config = use_chief_in_tf_config,
        distribution_strategy_type = distribution_strategy_type)
    args.extend(common_args)    
    
    package_args = [
        "--",
        "--job-dir=%s" % job_dir, 
        "--module-name=%s.%s" % (package_name, module_name),
        "--function-name=%s" % function_name]
    args.extend(package_args)
    
    if distribution_strategy_type:
        args.append("--distribution-strategy-type=%s" % distribution_strategy_type)
    
    print('running training job using package on Google Cloud Platform AI:')
    print(' '.join(args).replace(' --', '\n --').replace('\n', ' \\ \n'))
    
    if not dry_run:
        return gcp_runner.local_runner.run_process(args)
    

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted ai_platform_constants.ipynb.
Converted ai_platform_runner.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted local_runner.ipynb.
Converted sample_code.ipynb.
